In [5]:
import pandas as pd
import numpy as np
import string
from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix
from nltk import pos_tag,word_tokenize

In [6]:
from sklearn.datasets import fetch_20newsgroups
category_list = ['comp.sys.mac.hardware','rec.autos','misc.forsale','sci.space','talk.politics.guns']
new_groups = fetch_20newsgroups(categories=category_list,remove=('headers','footers','quotes'))

In [7]:
doc = new_groups.data
doc_df= pd.DataFrame(doc,columns={'Document':0})
doc_df.loc[0][0]

"\n740 Turbo in UK was good for 124mph. Useful for blowing away VW Beetles, though I\nbelieve the Beetle corners better. \n\nI can say without any doubt that I have never been blown away by any Volvo, ever.\nI've been blocked into a few car parks though by shit-head Volvo owners who 'only thought they'd be a few minutes'. This does not happen with the owners of any other makes of car."

In [8]:
doc_df['clean_text']=doc_df['Document'].str.lower().str.replace("[^a-z]"," ")
doc_df.head()

,Document,clean_text
0,\n740 Turbo in UK was good for 124mph. Useful ...,turbo in uk was good for mph useful f...
1,I am looking for a working docking deck (deck ...,i am looking for a working docking deck deck ...
2,\n >In article <1993Apr19.020359.26996@sq.sq.c...,in article apr sq sq co...
3,"A few posts back, somebody mentioned that the ...",a few posts back somebody mentioned that the ...
4,"\n\n\n\n\n\nIt still applies, except the astro...",it still applies except the astronomy t...


In [11]:
stop = set(stopwords.words('english'))

def sw(text):
    text = [word for word in text.split() if word not in stop] # spliting the words with space
    return " ".join(text) #

doc_df['Final'] = doc_df['clean_text'].apply(sw)
doc_df['Final'].head()

0    turbo uk good mph useful blowing away vw beetl...
1    looking working docking deck deck goes back ca...
2    article apr sq sq com msb sq sq com mark brade...
3    posts back somebody mentioned duo might crash ...
4    still applies except astronomy days long basel...
Name: Final, dtype: object

In [12]:
def myfun(text):
    a = word_tokenize(text)
    nltk.pos_tag(a)
    nouns = []
    is_noun = lambda x:x[:2] == 'NN'
    noun = [word for (word,x) in pos_tag(a) if is_noun(x)]
    nouns.extend(noun)
    return nouns

In [13]:
lst=[]
for i in doc_df['Final']:
    x = myfun(i)
    lst.extend(x)
lst   

['turbo',
 'mph',
 'beetles',
 'corners',
 'doubt',
 'car',
 'parks',
 'head',
 'volvo',
 'owners',
 'minutes',
 'owners',
 'car',
 'deck',
 'deck',
 'camera',
 'jvc',
 'gx',
 'tube',
 'video',
 'camera',
 'format',
 'please',
 'message',
 'anything',
 'decks',
 'video',
 'equipment',
 'sale',
 'consumer',
 'thank',
 'freedom',
 'van',
 'freedom',
 'bbs',
 'article',
 'apr',
 'sq',
 'sq',
 'com',
 'msb',
 'sq',
 'sq',
 'com',
 'mark',
 'brader',
 'mb',
 'mb',
 'figure',
 'anything',
 'jg',
 'sorry',
 'perijoves',
 'language',
 'periapsis',
 'apoapsis',
 'posts',
 'somebody',
 'duo',
 'kind',
 'non',
 'self',
 'duo',
 'crashes',
 'software',
 'kind',
 'ram',
 'problem',
 'battery',
 'connection',
 'thanks',
 'advance',
 'days',
 'radio',
 'astronomy',
 'gps',
 'data',
 'observatory',
 'others',
 'source',
 'data',
 'studies',
 'dynamics',
 'rotation',
 'purturbations',
 'time',
 'second',
 'year',
 'science',
 'vlbi',
 'track',
 'gallileo',
 'earth',
 'fuel',
 'afford',
 'observe',
 'id

In [14]:
unigram = CountVectorizer()
unigram.fit(lst)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [15]:
DTM = unigram.fit_transform(lst)
DTM

<123458x17227 sparse matrix of type '<class 'numpy.int64'>'
	with 118448 stored elements in Compressed Sparse Row format>

In [23]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components= 5, max_iter=10,random_state =0 )
lda_output = lda.fit_transform(DTM)
print(lda)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='batch', learning_offset=10.0,
                          max_doc_update_iter=100, max_iter=10,
                          mean_change_tol=0.001, n_components=5, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)


In [24]:
#Documnet  -topic matrix

topic_name = ['Topic '+str(i) for i in range(lda.n_components)]
doc_name = [ 'Document '+str(i) for i in range(DTM.shape[0])]

In [25]:
df_document_topic = pd.DataFrame(np.round(lda_output,2),columns=topic_name,index=doc_name)
df_document_topic.head()

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4
Document 0,0.1,0.1,0.6,0.1,0.1
Document 1,0.1,0.1,0.6,0.1,0.1
Document 2,0.1,0.6,0.1,0.1,0.1
Document 3,0.1,0.1,0.1,0.1,0.6
Document 4,0.1,0.6,0.1,0.1,0.1


In [26]:
#Creating the data frame between topic and terms

df_topic_keywords = pd.DataFrame(lda.components_,)


#Naming the columns and rows 

df_topic_keywords.columns= unigram.get_feature_names()
df_topic_keywords.index = topic_name
df_topic_keywords

,aa,aaa,aaah,aacvkc,aad,aaef,aalm,aamir,aamrl,aan,...,zwrm,zx,zxf,zxm,zxqi,zy,zyg,zyhszg,zyxel,zz
Topic 0,0.200034,0.200039,1.199815,0.200045,0.200044,0.200044,0.200044,0.200035,0.200038,0.200035,...,0.200047,7.199856,0.200045,0.200045,0.200047,0.200038,0.200044,0.200044,0.200038,3.199847
Topic 1,7.199862,0.200040,0.200047,0.200046,1.199823,0.200045,1.199823,0.200037,2.199846,0.200037,...,0.200049,0.200037,0.200046,0.200046,0.200049,2.199846,0.200045,1.199823,0.200039,0.200039
Topic 2,0.200033,0.200037,0.200044,0.200043,0.200043,0.200043,0.200043,0.200034,0.200037,0.200034,...,1.199808,0.200034,0.200043,0.200043,1.199808,0.200037,0.200043,0.200043,0.200037,0.200036
Topic 3,0.200035,2.199844,0.200047,1.199820,0.200045,0.200045,0.200045,0.200036,0.200039,0.200036,...,0.200048,0.200036,1.199820,1.199820,0.200048,0.200039,0.200045,0.200045,0.200039,0.200038
Topic 4,0.200036,0.200040,0.200047,0.200046,0.200046,1.199823,0.200046,4.199858,0.200040,4.199858,...,0.200049,0.200037,0.200046,0.200046,0.200049,0.200040,1.199823,0.200046,2.199847,0.200039


In [27]:
#finding the top 20 words for each topic

def show_topic(unigram, lda, n_words):
    keywords = np.array(unigram.get_feature_names())
    topic_keywords = []
    for topic_weights in lda.components_:
        topic_keywords_loc = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(topic_keywords_loc))
    return topic_keywords
topic_keywords = show_topic(unigram= unigram , lda =lda ,n_words = 20)

In [28]:
topic_keywords

[array(['power', 'price', 'things', 'law', 'state', 'work', 'firearms',
        'day', 'get', 'world', 'list', 'speed', 'problems', 'model', 'lot',
        'earth', 'fact', 'defense', 'auto', 'moon'], dtype='<U33'),
 array(['time', 'use', 'way', 'apple', 'case', 'mac', 'weapons', 'card',
        'cd', 'monitor', 'place', 'line', 'mission', 'rate', 'station',
        'center', 'idea', 'questions', 'flight', 'issue'], dtype='<U33'),
 array(['space', 'system', 'gun', 'year', 'problem', 'anyone', 'data',
        'edu', 'something', 'information', 'government', 'number',
        'please', 'thing', 'cars', 'bit', 'anything', 'money', 'disk',
        'condition'], dtype='<U33'),
 array(['people', 'thanks', 'control', 'sale', 'mail', 'software', 'com',
        'someone', 'program', 'file', 'part', 'cost', 'order', 'course',
        'times', 'news', 'mb', 'article', 'systems', 'support'],
       dtype='<U33'),
 array(['car', 'drive', 'years', 'point', 'guns', 'offer', 'computer',
        'memor

In [29]:
Top_20 = pd.DataFrame(topic_keywords,index=topic_name,columns=np.arange(1,21))#columns=topic_name,
Top_20.head()

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
Topic 0,power,price,things,law,state,work,firearms,day,get,world,list,speed,problems,model,lot,earth,fact,defense,auto,moon
Topic 1,time,use,way,apple,case,mac,weapons,card,cd,monitor,place,line,mission,rate,station,center,idea,questions,flight,issue
Topic 2,space,system,gun,year,problem,anyone,data,edu,something,information,government,number,please,thing,cars,bit,anything,money,disk,condition
Topic 3,people,thanks,control,sale,mail,software,com,someone,program,file,part,cost,order,course,times,news,mb,article,systems,support
Topic 4,car,drive,years,point,guns,offer,computer,memory,launch,engine,bill,orbit,technology,research,others,dos,vehicle,life,hardware,box


In [33]:
from sklearn.model_selection import GridSearchCV

search_params = {'n_components':[2,3,4,5,6,7,8,10]}

lda = LatentDirichletAllocation()

model = GridSearchCV(lda,param_grid=search_params)

model.fit(DTM)

C:\Users\Administrator\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LatentDirichletAllocation(batch_size=128,
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1,
                                                 learning_decay=0.7,
                                                 learning_method='batch',
                                                 learning_offset=10.0,
                                                 max_doc_update_iter=100,
                                                 max_iter=10,
                                                 mean_change_tol=0.001,
                                                 n_components=10, n_jobs=None,
                                                 perp_tol=0.1,
                                                 random_state=None,
                                                 topic_word_prior=None,
                                   

In [ ]:
best_lda_model = model.best_estimator_